# Kannada-MNIST CNN

## 1: Train Model & Prepare Case Base

In [1]:
import os
import cv2
import pandas as pd
import numpy as np
from numpy.core.numeric import full
import pickle
import time
import random 
import pprint
import warnings
from collections import Counter
from scipy.spatial import distance
from operator import itemgetter


from copy import deepcopy

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN


import skimage
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error

from skimage.color import gray2rgb, rgb2gray, label2rgb

import tensorflow as tf
tf.config.run_functions_eagerly(True)

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input, UpSampling2D, BatchNormalization, Activation, Reshape, Conv2DTranspose, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical



# from keras.layers import Input, Dense, Activation, Lambda, Reshape, Flatten
# from keras.layers import BatchNormalization
# from keras import losses
# from keras import backend as K
# from keras.losses import mse
# from keras.optimizers import Adam

import tensorflow as tf



# LIME
import lime
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.segmentation import mark_boundaries, slic
from scipy import ndimage

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

from PIL import Image


2023-11-09 08:59:30.611803: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load data
def load(f):
    return np.load(f)['arr_0']

# Load the data
X_train = load('X_kannada_MNIST_train.npz')
X_test = load('X_kannada_MNIST_test.npz')
y_train = load('y_kannada_MNIST_train.npz')
y_test = load('y_kannada_MNIST_test.npz')

oh_y_train = to_categorical(y_train)
oh_y_test = to_categorical(y_test)

# Normalize the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Reshape the data
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

# Output the shapes
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
#check tensorflow version
print(tf.__version__)

2.11.0


In [4]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.losses import CategoricalCrossentropy

model = Sequential()


# Cell 1 - 28x28x1
model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), padding='same', data_format='channels_last'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))


# Cell 2 - 32x4x14
model.add(Conv2D(64, (3, 3), padding='same', data_format='channels_last'))  # Removed the redundant input_shape
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_last'))

# Output - 64x7x7
model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(50))
model.add(Activation("relu"))
model.add(Dropout(0.2))
# model.add(Dense(10))
# model.add(Activation("softmax"))

logits = Dense(10)(model.output)  # This will give the raw logits

# Applying the softmax activation separately
predictions = Activation("softmax")(logits)

# Now, form the main model
model = Model(inputs=model.input, outputs=predictions)

# Form the logits model
logits_model = Model(inputs=model.input, outputs=logits)

model.compile(loss=CategoricalCrossentropy(),
              optimizer=Adam(),
              metrics=['accuracy'])

# Assuming that your data is in variables X_train, oh_y_train, X_test, oh_y_test
model.fit(X_train, oh_y_train,
          batch_size=256,
          epochs=2,
          verbose=1,
          validation_data=(X_test, oh_y_test))

# evaluate the model
scores = model.evaluate(X_train, oh_y_train)
print("Training Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

scores = model.evaluate(X_test, oh_y_test)
print("Test Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


# Save the model
model.save("NN5.h5")

2023-11-09 08:59:38.154760: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/2
235/235 [==============================] - 54s 227ms/step - loss: 0.2727 - accuracy: 0.9177 - val_loss: 0.1605 - val_accuracy: 0.9524
Epoch 2/2
1875/1875 [==============================] - 31s 16ms/step - loss: 0.0290 - accuracy: 0.9915
Training Set: 
accuracy: 99.15%
313/313 [==============================] - 5s 15ms/step - loss: 0.1347 - accuracy: 0.9618
Test Set: 
accuracy: 96.18%


In [5]:
knn_X_train = X_train.reshape(X_train.shape[0], 28*28)
knn_X_test = X_test.reshape(X_test.shape[0], 28*28)

In [6]:
# Train final k-NN 
knn_clf = KNeighborsClassifier(n_neighbors=1, algorithm="brute") 
knn_clf.fit(knn_X_train, y_train)

KNeighborsClassifier(algorithm='brute', n_neighbors=1)

In [7]:
# Check the accuracy on this particular split to make sure that it is not too far removed from k-fold.
knn_predictions_test = knn_clf.predict(knn_X_test)
print("k-NN Accuracy Test:", accuracy_score(y_test, knn_predictions_test))

k-NN Accuracy Test: 0.9198


In [8]:
nn_pred = model.predict_classes(X_test)

right = 0
for i in range(len(nn_pred)):
    if knn_predictions_test[i] == nn_pred[i]:
        right += 1
print("Agreement:", right/len(nn_pred))

AttributeError: 'Functional' object has no attribute 'predict_classes'

In [ ]:
# Check confusion matrix kNN
confusion_matrix(y_test, knn_predictions_test, labels=None, sample_weight=None)

array([[862, 121,   2,   3,   2,   0,   0,   3,   3,   4],
       [ 18, 971,   0,   3,   1,   0,   0,   2,   2,   3],
       [  2,   2, 991,   1,   0,   0,   3,   0,   0,   1],
       [ 14,  12,   2, 925,   8,   8,   2,  28,   0,   1],
       [  0,   0,   0,   5, 974,   4,   5,   3,   2,   7],
       [  0,  10,   7,  17, 110, 828,   3,  11,   6,   8],
       [  2,   1,   1,   6,   8,   6, 918,  39,   1,  18],
       [  8,   4,   1,  19,  24,   2, 105, 822,   2,  13],
       [ 35,   5,   1,   0,   1,   0,   0,   0, 955,   3],
       [  6,  11,   0,   0,  10,   1,   8,   4,   8, 952]])

In [ ]:
# Check confusion matrix NN 
confusion_matrix(y_test, model.predict_classes(X_test), labels=None, sample_weight=None)

array([[936,  55,   0,   0,   2,   0,   0,   1,   5,   1],
       [  2, 995,   0,   2,   0,   0,   0,   0,   0,   1],
       [  2,   0, 996,   0,   0,   0,   0,   1,   0,   1],
       [  0,   8,   1, 949,  10,  11,   6,  14,   0,   1],
       [  0,   0,   1,   0, 991,   1,   1,   3,   1,   2],
       [  0,   1,   1,   0,  46, 952,   0,   0,   0,   0],
       [  0,   1,   0,   0,   4,   1, 975,   1,   2,  16],
       [  9,   1,   2,   8,   4,   0,  42, 928,   2,   4],
       [  0,   0,   0,   0,   0,   0,   1,   0, 997,   2],
       [  0,   0,   0,   0,   0,   0,   1,   0,   3, 996]])

In [ ]:
# Save the CBR model to disk
pickle.dump(knn_clf, open('k-nn_model.sav', 'wb'))

In [ ]:
# Save Keras Models to disk
model.save("NN.h5")

In [ ]:
# Save Dataframes
np.save("X_train", X_train)
np.save("X_test", X_test)
np.save("y_train", y_train)
np.save("y_test", y_test)

np.save("knn_X_train", knn_X_train)
np.save("knn_X_test", knn_X_test)

np.save("oh_y_train", oh_y_train)
np.save("oh_y_test", oh_y_test)